In [3]:
!pip install --no-index h5py

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/x86-64-v3, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
import os

# Add the parent directory to the Python path
script_dir = os.path.dirname(os.getcwd())  # Get the directory where the script is located
parent_dir = os.path.dirname(script_dir)  # Get the parent directory

print(f"{script_dir = }")
print(f"{parent_dir = }")

sys.path.append(parent_dir)

script_dir = '/lustre06/project/6067616/soroush1/vitalab-trianing-clean-code/notebooks'
parent_dir = '/lustre06/project/6067616/soroush1/vitalab-trianing-clean-code'


In [6]:
from lit_modules.datamodule import MuriDataModule
from argparse import Namespace

import lightning as L

2024-09-10 15:14:14,481 - INFO - ImageNet module loaded.
2024-09-10 15:14:14,886 - WARNING - Note: 00050591.jpg is removed because it can't be loaded in ffcv.


In [7]:
# Define hyperparameters
hparams = Namespace(
    data_dir="/scratch/soroush1/memorability/muri1320",
    image_size=224,
    batch_size=128,
    num_workers=4,
    change_labels = False,
    pin_memories=[False, False, False],  # [train, val, test]
    return_paths = True
)

# Create the DataModule
data_module = MuriDataModule(hparams)

# Prepare data and setup
data_module.prepare_data()
data_module.setup("test")

# Print dataset sizes
print(f"test dataset size: {len(data_module.test_dataset)}")

test_dl = data_module.test_dataloader()

Muri data not found. Please download the dataset manually.
Test dataset size: 1320
test dataset size: 1320


In [8]:
from lit_modules.modules import ModelLightning

# Define hyperparameters for the model
num_classes = 1000
task_type = "classification"

model_hparams = {
    "arch": "alexnet",
    "use_blurpool": True,
    "pretrained": False,
    "num_classes": num_classes,
    "lr": 0.0001,
    "weight_decay": 1e-4,
    "momentum": 0.9,
    "nesterov": True,
    "norm_mean": [0.485, 0.456, 0.406],
    "norm_std": [0.229, 0.224, 0.225],
    "task_type": task_type,
    "experiment": "one",
    "optimizer": "sgd",
    "scheduler": "plateau",
    "step_size": 30,
    "max_epochs": 100,
    "random_training": False,
    "use_ckpt": True,
    "checkpoint": "/home/soroush1/projects/def-kohitij/soroush1/vitalab-trianing-clean-code/experiments/alexnet/classification/version_2/checkpoints/epoch=49-step=125150.ckpt",
}

model = ModelLightning(model_hparams)

2024-09-10 15:14:15,734 - INFO - Creating model with architecture: alexnet
2024-09-10 15:14:15,734 - INFO - Pretrained: False
2024-09-10 15:14:15,957 - INFO - Modifying model for classification task
2024-09-10 15:14:15,972 - INFO - Applying BlurPool to the model
2024-09-10 15:14:15,973 - INFO - Model creation completed
2024-09-10 15:14:16,054 - INFO - Loading model from checkpoint: /home/soroush1/projects/def-kohitij/soroush1/vitalab-trianing-clean-code/experiments/alexnet/classification/version_2/checkpoints/epoch=49-step=125150.ckpt
2024-09-10 15:14:17,392 - INFO - Weights for features.0.weight have been updated.
2024-09-10 15:14:17,393 - INFO - Weights for features.0.bias have been updated.
2024-09-10 15:14:17,409 - INFO - Weights for features.3.weight have been updated.
2024-09-10 15:14:17,409 - INFO - Weights for features.3.bias have been updated.
2024-09-10 15:14:17,411 - INFO - Weights for features.6.weight have been updated.
2024-09-10 15:14:17,411 - INFO - Weights for features

Set up components for classification task.
Loading model from checkpoint: epoch=49-step=125150.ckpt


In [9]:
model = model.get_model_for_feature_extraction(it_only=True)

2024-09-10 15:14:17,711 - INFO - Trainable nodes: ['x', 'features.0', 'features.1', 'features.2', 'features.3', 'features.4', 'features.5', 'features.6', 'features.7', 'features.8', 'features.9', 'features.10', 'features.11', 'features.12', 'avgpool', 'flatten', 'classifier.0', 'classifier.1', 'classifier.2', 'classifier.3', 'classifier.4', 'classifier.5', 'classifier.6']
2024-09-10 15:14:17,712 - INFO - Evaluatable nodes: ['x', 'features.0', 'features.1', 'features.2', 'features.3', 'features.4', 'features.5', 'features.6', 'features.7', 'features.8', 'features.9', 'features.10', 'features.11', 'features.12', 'avgpool', 'flatten', 'classifier.0', 'classifier.1', 'classifier.2', 'classifier.3', 'classifier.4', 'classifier.5', 'classifier.6']
2024-09-10 15:14:17,712 - INFO - Desired layers: features.12


In [16]:
import torch
import re
from tqdm import tqdm
import h5py
from lit_modules.modules.utils import InferioTemporalLayer

def sort_batch_by_filename(tensor, filenames):
    # Extract indices from filenames
    indices = [int(re.search(r'(\d+)', fname).group()) for fname in filenames]
    
    # Create a list of (index, tensor_slice, filename) tuples
    indexed_data = list(zip(indices, tensor, filenames))
    
    # Sort the list based on the extracted indices
    sorted_data = sorted(indexed_data, key=lambda x: x[0])
    
    # Unzip the sorted list
    _, sorted_tensor, sorted_filenames = zip(*sorted_data)
    
    # Stack the tensor slices back into a single tensor
    sorted_tensor = torch.stack(sorted_tensor)
    
    return sorted_tensor, sorted_filenames

def extract_and_concatenate_features(model, test_dl, arch):
    all_outputs = []
    all_img_paths = []
    for img, img_path in tqdm(test_dl):
        with torch.no_grad():  # Disable gradient calculation
            # logger.info(f"Using input size: {img.size()}")
            output = model(img)[getattr(InferioTemporalLayer, arch.upper()).value]  # extract IT layer
            output, img_path = sort_batch_by_filename(output, img_path)
        
        all_outputs.append(output.cpu())  # Move to CPU to save GPU memory
        all_img_paths.extend(img_path)
        
        # Delete batch data to free up memory
        del img, output
        torch.cuda.empty_cache()  # Clear CUDA cache
    
    # Concatenate all outputs into a single tensor
    concatenated_output = torch.cat(all_outputs, dim=0)
    
    # Delete all_outputs to free up memory
    del all_outputs
    torch.cuda.empty_cache()
    
    # Reshape the output to (samples, -1)
    reshaped_output = concatenated_output.view(concatenated_output.size(0), -1)
    
    # Convert to numpy and delete the torch tensor
    reshaped_output = reshaped_output.detach().numpy()
    del concatenated_output
    torch.cuda.empty_cache()
    
    return reshaped_output, all_img_paths

def save_h5(data, model_name, task_name, dst_path: str, layer_name: str = "it", ):
    """
    Load features from a pickle file and save them to an HDF5 file.

    Args:
    model_name (str): Name of the model used to extract features.
    task_name (str): Name of the task or dataset.
    layer_name (str): Name of the layer from which features were extracted.
    dst_path (str): Directory to save the HDF5 file.

    Returns:
    None
    """

    print(
        f"Model: {model_name}, Task: {task_name}, Layer: {layer_name}, Shape: {data.shape}"
    )

    # Ensure the destination directory exists
    os.makedirs(dst_path, exist_ok=True)

    # Construct the full path for the HDF5 file
    h5_file = os.path.join(dst_path, f"{model_name}_{task_name}_1.h5")

    # Save the NumPy array as a .h5 file
    with h5py.File(h5_file, "w") as hf:
        hf.create_dataset("features", data=data)

    print(f"Features saved to {h5_file}")


def get_config(model_name: str, checkpoint: str):
    num_classes = 1000
    task_type = "classification"
    return {
    "arch": model_name,
    "use_blurpool": True,
    "pretrained": False,
    "num_classes": num_classes,
    "lr": 0.0001,
    "weight_decay": 1e-4,
    "momentum": 0.9,
    "nesterov": True,
    "norm_mean": [0.485, 0.456, 0.406],
    "norm_std": [0.229, 0.224, 0.225],
    "task_type": task_type,
    "experiment": "one",
    "optimizer": "sgd",
    "scheduler": "plateau",
    "step_size": 30,
    "max_epochs": 100,
    "random_training": False,
    "use_ckpt": True,
    "checkpoint": checkpoint,
    }

def get_data(input_size: int):
    # Define hyperparameters
    hparams = Namespace(
        data_dir="/scratch/soroush1/memorability/muri1320",
        image_size=input_size,
        batch_size=128,
        num_workers=4,
        change_labels = False,
        pin_memories=[False, False, False],  # [train, val, test]
        return_paths = True
    )
    
    # Create the DataModule
    data_module = MuriDataModule(hparams)
    
    # Prepare data and setup
    data_module.prepare_data()
    data_module.setup("test")
    
    # Print dataset sizes
    print(f"test dataset size: {len(data_module.test_dataset)}")
    
    return data_module.test_dataloader()

In [11]:
# Extract IT activation
model_name = "alexnet"

concatenated_features, all_image_paths = extract_and_concatenate_features(model, test_dl, model_name)

print(f"Concatenated features shape: {concatenated_features.shape}")
print(f"Total number of images: {len(all_image_paths)}")

task_name = "imagenet_shuffle"
dst_dir = "./features"

# Call the save_h5 function
save_h5(concatenated_features, model_name, task_name, dst_dir)

# Load the saved data and compare
expected_file = os.path.join(dst_dir, f"{model_name}_{task_name}_1.h5")
with h5py.File(expected_file, "r") as hf:
    saved_data = hf["features"][:]

saved_data.shape

100%|██████████| 11/11 [00:25<00:00,  2.36s/it]


Concatenated features shape: (1320, 12544)
Total number of images: 1320
Model: alexnet, Task: imagenet_shuffle, Layer: it, Shape: (1320, 12544)
Features saved to ./features/alexnet_imagenet_shuffle_1.h5


(1320, 12544)

In [18]:
import torch
import gc
import logging
from datetime import datetime
from lit_modules.modules import ModelLightning
from lit_modules.datamodule import MuriDataModule
from argparse import Namespace

import lightning as L


# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

configs = {
    "alexnet": {
        "imagenet_shuffle": "/home/soroush1/projects/def-kohitij/soroush1/vitalab-trianing-clean-code/checkpoints/classification/alexnet/epoch=71-step=180216.ckpt"
    },
    "efficientnet_b0": {
        "imagenet_shuffle": "/home/soroush1/projects/def-kohitij/soroush1/vitalab-trianing-clean-code/checkpoints/classification/efficientnet_b0/epoch=38-step=195195.ckpt"
    },
    "inception_v3": {
        "imagenet_shuffle": ""
    },
    "vit_b_16": {
        "imagenet_shuffle": "/home/soroush1/projects/def-kohitij/soroush1/vitalab-trianing-clean-code/checkpoints/classification/vit_b_16/epoch=8-step=45045.ckpt"
    },
    "vit_b_32": {
        "imagenet_shuffle": "/home/soroush1/projects/def-kohitij/soroush1/vitalab-trianing-clean-code/checkpoints/classification/vit_b_32/epoch=31-step=40064.ckpt"
    }
}

dst_dir = "./features"

logger.info("Starting processing for all models and tasks")

for model_name, task_checkpoints in configs.items():
    for task, checkpoint in task_checkpoints.items():

        if checkpoint == "":
            continue
        
        logger.info(f"Processing model: {model_name}, task: {task}")
        
        start_time = datetime.now()
        
        # Determine input size based on model
        input_size = 224 if model_name in ["vit_b_16", "vit_b_32"] else 256
        # input_size = 256
        
        logger.info(f"Using input size: {input_size}")
        
        # Get data
        logger.info("Getting data...")
        data = get_data(input_size=input_size)
        logger.info(f"Data loaded. Shape: {data.shape if hasattr(data, 'shape') else 'N/A'}")
        
        # Get model configuration
        logger.info("Getting model configuration...")
        hparams = get_config(model_name, checkpoint)
        logger.info("Model configuration loaded")
        
        # Initialize and prepare the model
        logger.info("Initializing and preparing the model...")
        model = ModelLightning(hparams)
        model = model.get_model_for_feature_extraction(it_only=True)
        logger.info("Model prepared for feature extraction")
        
        # Extract features
        logger.info("Extracting features...")
        concatenated_features, all_image_paths = extract_and_concatenate_features(model, data, model_name)
        logger.info(f"Features extracted. Shape: {concatenated_features.shape}")
        
        # Save features
        logger.info("Saving features...")
        save_h5(concatenated_features, model_name, task, dst_dir)
        logger.info(f"Features saved to {dst_dir}")
        
        # Clean up to free memory
        logger.info("Cleaning up memory...")
        del model
        del data
        del concatenated_features
        del all_image_paths
        torch.cuda.empty_cache()  # Clear CUDA cache if using GPU
        gc.collect()  # Run garbage collector
        logger.info("Memory cleanup completed")
        
        end_time = datetime.now()
        duration = end_time - start_time
        logger.info(f"Processing completed for {model_name} - {task}. Duration: {duration}")

logger.info("Processing completed for all models and tasks")

2024-09-10 15:42:06,929 - INFO - Starting processing for all models and tasks
2024-09-10 15:42:06,930 - INFO - Processing model: alexnet, task: imagenet_shuffle
2024-09-10 15:42:06,930 - INFO - Using input size: 256
2024-09-10 15:42:06,931 - INFO - Getting data...
2024-09-10 15:42:06,934 - INFO - Data loaded. Shape: N/A
2024-09-10 15:42:06,934 - INFO - Getting model configuration...
2024-09-10 15:42:06,934 - INFO - Model configuration loaded
2024-09-10 15:42:06,935 - INFO - Initializing and preparing the model...
2024-09-10 15:42:06,935 - INFO - Creating model with architecture: alexnet
2024-09-10 15:42:06,936 - INFO - Pretrained: False


Muri data not found. Please download the dataset manually.
Test dataset size: 1320
test dataset size: 1320


2024-09-10 15:42:07,224 - INFO - Modifying model for classification task
2024-09-10 15:42:07,244 - INFO - Applying BlurPool to the model
2024-09-10 15:42:07,245 - INFO - Model creation completed
2024-09-10 15:42:07,264 - INFO - Loading model from checkpoint: /home/soroush1/projects/def-kohitij/soroush1/vitalab-trianing-clean-code/checkpoints/classification/alexnet/epoch=71-step=180216.ckpt
2024-09-10 15:42:07,399 - INFO - Weights for features.0.weight have been updated.
2024-09-10 15:42:07,400 - INFO - Weights for features.0.bias have been updated.
2024-09-10 15:42:07,404 - INFO - Weights for features.3.weight have been updated.
2024-09-10 15:42:07,405 - INFO - Weights for features.3.bias have been updated.
2024-09-10 15:42:07,408 - INFO - Weights for features.6.weight have been updated.
2024-09-10 15:42:07,408 - INFO - Weights for features.6.bias have been updated.
2024-09-10 15:42:07,411 - INFO - Weights for features.8.weight have been updated.
2024-09-10 15:42:07,411 - INFO - Weight

Set up components for classification task.
Loading model from checkpoint: epoch=71-step=180216.ckpt


100%|██████████| 11/11 [00:06<00:00,  1.76it/s]
2024-09-10 15:42:13,933 - INFO - Features extracted. Shape: (1320, 12544)
2024-09-10 15:42:13,934 - INFO - Saving features...
2024-09-10 15:42:14,043 - INFO - Features saved to ./features
2024-09-10 15:42:14,044 - INFO - Cleaning up memory...


...

Set up components for classification task.
Loading model from checkpoint: epoch=31-step=40064.ckpt


100%|██████████| 11/11 [01:48<00:00,  9.83s/it]
2024-09-10 15:51:39,351 - INFO - Features extracted. Shape: (1320, 38400)
2024-09-10 15:51:39,352 - INFO - Saving features...


Model: vit_b_32, Task: imagenet_shuffle, Layer: it, Shape: (1320, 38400)


2024-09-10 15:51:39,766 - INFO - Features saved to ./features
2024-09-10 15:51:39,767 - INFO - Cleaning up memory...
2024-09-10 15:51:39,911 - INFO - Memory cleanup completed
2024-09-10 15:51:39,912 - INFO - Processing completed for vit_b_32 - imagenet_shuffle. Duration: 0:01:51.413093
2024-09-10 15:51:39,912 - INFO - Processing completed for all models and tasks


Features saved to ./features/vit_b_32_imagenet_shuffle_1.h5
